# SIR Model on Traffic and its Application

## Simple SIR Model

In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# ipywidgets currently only works with jupyter notebook, not lab
# installation guide: https://ipywidgets.readthedocs.io/en/stable/user_install.html
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive
from IPython.display import display
%matplotlib inline

In [4]:
def plot_simple_SIR(beta=0.2, gamma=0.1):
    # Total population, N.
    N = 1000
    # Initial number of infected and recovered individuals, I0 and R0.
    I0, R0 = 1, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    # beta, gamma = 0.2, 1./10 
    # A grid of time points (in days)
    t = np.linspace(0, 160, 160)

    # The SIR model differential equations.
    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt

    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
    ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
    ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
    ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
    ax.set_xlabel('Time /days')
    ax.set_ylabel('Number (1000s)')
    ax.set_ylim(0,1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(visible=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

In [5]:
# Interactive beta and gamma
# beta: effective contact rate
# gamma: mean recovery rate, where 1/γ is 
#        the mean period of time during which 
#        an infected individual can pass it on

interactive_plot = interactive(plot_simple_SIR, beta=(0.0, 1.0, 0.01), gamma=(0.0, 1.0, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.2, description='beta', max=1.0, step=0.01), FloatSlider(value=0.1, d…

In [6]:
def plot_fremont_SIR(beta=0.25, gamma=0.05):
    # Total population, N.
    N = 4397
    # Initial number of infected and recovered individuals, I0 and R0.
    # rho = 0.9
    I0, R0 = 450, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    # beta, gamma = 0.2, 1./10 
    # A grid of time points (in quarters of an hour)
    t = np.linspace(0, 24, 24)

    # The SIR model differential equations.
    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt

    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
    ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
    ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
    ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
    ax.set_xlabel('Time /days')
    ax.set_ylabel('Number (1000s)')
#     ax.set_ylim(0,1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(visible=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

In [7]:
# Interactive beta and gamma
# beta: effective contact rate
# gamma: mean recovery rate, where 1/γ is 
#        the mean period of time during which 
#        an infected individual can pass it on

interactive_plot = interactive(plot_fremont_SIR, beta=(0.0, 1.0, 0.01), gamma=(0.0, 1.0, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.25, description='beta', max=1.0, step=0.01), FloatSlider(value=0.05,…

## Building the Traffic SIR Model (Saberi, M. et al.)

In [8]:
def congested(v_t, v_max, rho):
    """
    Return whether the link is congested
    
    Parameters:
        v_t: the speed of the link at time t
        v_max: the speed limit of the link
        rho (float): congestion threshold
        
    Returns:
        congested (int): 1 if congested else 0
    """
    congested = (v_t / v_max) < rho
    return congested

In [9]:
def propagation_time(l, q_c, v_f, k_j):
    """
    Return the time for congestion to propagate from link i
    to the upstream link (i + 1)
    
    Parameters:
        l: the length of the link
        q_c: maximum flow of the link
        v_f : free fkiw speed
        k_j: jam density
        
    Returns:
        eta (float): propagation time
    """
    # critical density
    k_c = q_c / v_f
    
    # shockwave speed
    omega = q_c / (k_j - k_c)
    
    # propagation time
    eta = l / omega
    
    return eta

In [10]:
def deriv_congested_links(l, q, v_f, k_j, t):
    eta = propagation_time(l, q[t], v_f, k_j)
    dCdt = 1 / eta
    return dCdt

In [13]:
def plot_traffic_SIR(beta=0.2, miu=0.1):
    # Number of directed links
    N = 1000
    # Initial number of congested links
    C_0 = 100
    # Initial number of recovered links
    R_0 = 0
    # Initial number of free flow links
    F_0 = N - C_0 - R_0
    # average effective contacts at upstream,
    # derived from homogenous mixing assumption
    k = 1 # needs adjacency matrix
    
    c_0, r_0, f_0 = C_0 / N, R_0 / N, F_0 / N

    # A grid of time points (in 360 mins)
    t = np.linspace(0, 360, 360)

    # The SIR model differential equations
    def deriv_traffic(y, t, N, beta, miu):
        """
        Return the ordinary differential equations (ODEs) at time t

        Parameters:
            y: the initial conditions [F, C, R]
            t: time steps
            N: total number of links
            beta: congestion propagation rate
            miu: congestion recovery rate

        Returns:
            the ODEs at time t
        """
        f, c, r = y

        dcdt = - miu * c + beta * k * c * (1 - r - c)
        drdt = miu * c
        dfdt = - beta * k * c * (1 - r - c)

        return dcdt, drdt, dfdt

    # Initial conditions vector
    y_0 = f_0, c_0, r_0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv_traffic, y_0, t, args=(N, beta, miu))
    F, C, R = ret.T

    # Plot the data on three separate curves for F(t), C(t) and R(t)
    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
    ax.plot(t, F/N, 'b', alpha=0.5, lw=2, label='Freeflow')
    ax.plot(t, C/N, 'r', alpha=0.5, lw=2, label='Congested')
    ax.plot(t, R/N, 'g', alpha=0.5, lw=2, label='Recovered')
    ax.set_xlabel('Time /mins')
    ax.set_ylabel('Number (1000s)')
    ax.set_ylim(0, 1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(visible=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

In [14]:
# Interactive beta and gamma
# beta: effective contact rate
# gamma: mean recovery rate, where 1/γ is 
#        the mean period of time during which 
#        an infected individual can pass it on

interactive_plot = interactive(plot_traffic_SIR, beta=(0.0, 1.0, 0.01), miu=(0.0, 1.0, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.2, description='beta', max=1.0, step=0.01), FloatSlider(value=0.1, d…

In [17]:
def plot_traffic_SIR(beta=0.2, miu=0.1):
    # Number of directed links
    N = 1000
    # Initial number of congested links
    C_0 = 1
    # Initial number of recovered links
    R_0 = 0
    # Initial number of free flow links
    F_0 = N - C_0 - R_0
    # average effective contacts at upstream,
    # derived from homogenous mixing assumption
    k = 1 # needs adjacency matrix

    # A grid of time points (in 360 mins)
    t = np.linspace(0, 360, 360)

    # The SIR model differential equations
    def deriv_traffic(y, t, N, beta, miu):
        """
        Return the ordinary differential equations (ODEs) at time t

        Parameters:
            y: the initial conditions [F, C, R]
            t: time steps
            N: total number of links
            beta: congestion propagation rate
            miu: congestion recovery rate

        Returns:
            the ODEs at time t
        """
        F, C, R = y
        frac_F, frac_C, frac_R = F / N, C / N, R / N

        dFdt = - beta * k * C * (1 - frac_R - frac_C)
        dCdt = - miu * C + beta * k * C * (1 - frac_R - frac_C)
        dRdt = miu * C
        
        return dCdt, dRdt, dFdt

    # Initial conditions vector
    y_0 = F_0, C_0, R_0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv_traffic, y_0, t, args=(N, beta, miu))
    F, C, R = ret.T

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
    ax.plot(t, F/N, 'b', alpha=0.5, lw=2, label='Freeflow')
    ax.plot(t, C/N, 'r', alpha=0.5, lw=2, label='Congested')
    ax.plot(t, R/N, 'g', alpha=0.5, lw=2, label='Recovered')
    ax.set_xlabel('Time /mins')
    ax.set_ylabel('Number (1000s)')
    ax.set_ylim(0, 1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(visible=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

In [18]:
# Interactive beta and gamma
# beta: effective contact rate
# gamma: mean recovery rate, where 1/γ is 
#        the mean period of time during which 
#        an infected individual can pass it on

interactive_plot = interactive(plot_traffic_SIR, beta=(0.0, 1.0, 0.01), miu=(0.0, 1.0, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.2, description='beta', max=1.0, step=0.01), FloatSlider(value=0.1, d…

## Extract data from the provided simulation (10/18)

In [10]:
import sqlite3
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import contextily as cx

In [14]:
# GLOBAL VARIABLES/CONSTANTS

# SQLite File Path formatter
__SQLITE_PATH_FORMAT = "Data/carl_10_18_18_saberi/Dynamel_06_09_2018_rpl_{number}.sqlite"

# sections.shp File Path
# __SECTION_SHP = "Data/sections.shp"

# Number of experiments
__NUM_EXP = 3

# SQL Query to be excecuted for MISECT table
__SQL_EXTRACT_QUERY = 'SELECT * FROM MESECT'

# Columns to extract from MISECT table
__COLUMNS = ['ent', 'eid', 'flow_capacity', 'speed', 'travel', 'traveltime']

In [15]:
# Create a SQL connection to our SQLite database

# A list of established connections to our databases
con = []

for i in range(__NUM_EXP):
    con.append(sqlite3.connect(__SQLITE_PATH_FORMAT.format(number=i)))

In [16]:
# Run SQL query and convert SQL to DataFrame

# List of the sql queries
sql_queries = []

# List of dataframes extracted from each experiment
df = []
for i in range(__NUM_EXP):
    # Run SQL
    query = pd.read_sql(__SQL_EXTRACT_QUERY, con[i])
    sql_queries.append(query)
    
    # Convert SQL to DataFrame
    dataframe = pd.DataFrame(query, columns = __COLUMNS)
    df.append(dataframe)

KeyboardInterrupt: 

In [ ]:
# Create a deep copy of df as back up in order not to rerun the above cell
df_copy = copy.deepcopy(df)